# Time-Series Forecasting with Amazon SageMaker Autopilot

### Contents

- [1. Introduction](#introduction)
- [2. Setup](#setup)
- [3. 学習](#training)
- [4. 予測](#forecast)
  - [4.1. リアルタイム推論を利用する場合](#realtime)
  - [4.2. バッチ推論 を利用する場合](#batch)

## 1. Introduction <a name='introduction'>

このノートブックでは、Amazon SageMaker Autopilot を使用して時系列モデルを作成し、時系列予測を生成する過程を体験できます。まず初めに S3 上に格納されている表形式の履歴データを使ってモデルを学習するための API を作成します。モデルを学習した後は、予測を生成する際にバッチ推論を実施するか、リアルタイムエンドポイントを作成したリアルタイム推論を実施するかをオプションとして選択することができます。</n></n>

モデルを作成する過程において、 SageMaker Autopilot は複数の時系列モデルを同時に作成します。これらのモデルはすべて、予測誤差を最小限に抑えるように複数の学習モデルをブレンドし、単一のアンサンブルモデルとして結合されます。結合された単一のアンサンブルモデルと作成された複数の学習モデルのメタデータとモデルの詳細をユーザーは入手することができます。SageMaker Autopilot はこの一連のプロセスをオーケストレーションし、いくつものアーティファクトをユーザーに提供します。

アーティファクトには以下が含まれます。 
- 複数の時間 Window にわたるモデルごとのバックテスト（ホールドアウト）テスト
- モデルごとの精度指標
- アンサンブルモデルのバックテスト結果と精度指標
- 各説明変数および静的メタデータの重要性を示す説明可能性レポート
- バッチ推論・リアルタイム推論に使用できる全てのモデルのアーティファクト

## 2. セットアップ <a name='setup'>

In [ ]:
# Update boto3 using this method, or your preferred method
!pip install --upgrade boto3 --quiet
!pip install --upgrade sagemaker --quiet
!pip install --upgrade stepfunctions --quiet

In [ ]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from time import gmtime, strftime, sleep
import datetime
import pandas as pd
import os

region = boto3.Session().region_name
session = sagemaker.Session()

# Modify the following default_bucket to use a bucket of your choosing
bucket = session.default_bucket()
#bucket = 'my-bucket'
prefix = 'demo-autopilot-forecast'

role = get_execution_role()

# This is the client we will use to interact with SageMaker Autopilot
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

本ノートブックでは以下のコードで出力したダミーのデータセットを使用します。サンプルのデータセットではなく、ご自身のデータセットを利用するようにノートブックの内容を変更することも可能です。
以下のコードで出力される CSV ファイルの構造は次のとおりです。
- time (必須: 日時)
- measure_name (必須：製品の唯一の属性を示す)
- region (静的：product_code のカテゴリを示す)
- sales (必須: 予測対象列)

In [ ]:
import random
import datetime
import csv
import math

# 開始日時と終了日時を設定
start_date = datetime.datetime(2021, 1, 1, 0, 0, 0)
end_date = datetime.datetime(2024, 4, 30, 23, 59, 59)


# 商品ごとの季節係数を設定
product_seasonal_factors = {
    'product_a': { 1: 0.85, 2: 0.90, 3: 1.00, 4: 1.03, 5: 1.06, 6: 1.10,7: 1.15, 8: 1.10, 9: 1.06, 10: 1.03, 11: 1.00, 12: 0.90},
    'product_b': {1: 1.0, 2: 1.0, 3: 1.1, 4: 1.2, 5: 1.3, 6: 1.4, 7: 1.5, 8: 1.4, 9: 1.3, 10: 1.2, 11: 1.1, 12: 1.0},
    'product_c': {1: 1.2, 2: 1.1, 3: 1.0, 4: 0.9, 5: 0.8, 6: 0.7, 7: 0.6, 8: 0.7, 9: 0.8, 10: 0.9, 11: 1.0, 12: 1.1}
}

## 商品ごとの曜日係数を設定
product_weekday_factors = {
    'product_a': {0: 1.0, 1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: 0.7, 6: 1.0},
    'product_b': {0: 1.2, 1: 1.0, 2: 0.9, 3: 0.9, 4: 1.0, 5: 1.1, 6: 1.3},
    'product_c': {0: 0.9, 1: 0.7, 2: 0.6, 3: 0.6, 4: 0.7, 5: 0.8, 6: 1.0}
}

# 商品ごとの時間帯係数を設定
product_hour_factors = {
    'product_a': {0: 0.05, 1: 0.05, 2: 0.05, 3: 0.05, 4: 0.05, 5: 0.05, 6: 0.1, 7: 0.2, 8: 0.3,9: 0.5, 10: 0.7, 11: 0.8, 12: 0.9, 13: 1.0, 14: 1.1, 15: 1.2, 16: 1.3, 17: 1.3, 18: 1.2, 19: 1.1, 20: 1.0, 21: 0.8, 22: 0.5, 23: 0.2},
    'product_b': {0: 0.2, 1: 0.2, 2: 0.2, 3: 0.2, 4: 0.2, 5: 0.3, 6: 0.4, 7: 0.6, 8: 0.8, 9: 1.0, 10: 1.1, 11: 1.2, 12: 1.3, 13: 1.4, 14: 1.5, 15: 1.6, 16: 1.7, 17: 1.8, 18: 1.7, 19: 1.6, 20: 1.4, 21: 1.2, 22: 0.9, 23: 0.6},
    'product_c': {0: 0.3, 1: 0.3, 2: 0.3, 3: 0.3, 4: 0.3, 5: 0.4, 6: 0.5, 7: 0.7, 8: 0.9, 9: 1.1, 10: 1.2, 11: 1.3, 12: 1.4, 13: 1.5, 14: 1.6, 15: 1.7, 16: 1.8, 17: 1.9, 18: 1.8, 19: 1.7, 20: 1.5, 21: 1.3, 22: 1.0, 23: 0.7}
}

# 場所ファクターを設定
location_factor = {
    'location_a': 1.2,
    'location_b': 1.0,
    'location_c': 0.8
}

# 1時間ごとのループ
current_date = start_date
sales_list = []
while current_date <= end_date:
    for product in ['product_a', 'product_b', 'product_c']:
        for location in ['location_a', 'location_b', 'location_c']:
            # 季節による売上係数を取得
            month = current_date.month
            seasonal_factor = product_seasonal_factors[product][month]

            # 曜日による売上係数を取得
            weekday = current_date.weekday()
            weekday_factor = product_weekday_factors[product][weekday]

            # 時間帯による売上係数を取得
            hour = current_date.hour
            hour_factor = product_hour_factors[product][hour]

            # 場所による売上係数を取得
            location_factor_value = location_factor[location]

            # 年単位の増加率を計算（年ごとに5~10%増加）
            year_factor = 1 + random.uniform(0.05, 0.1) * (current_date.year - start_date.year)

            # 基本売上を設定
            base_sales = 100 * seasonal_factor * weekday_factor * hour_factor * year_factor * location_factor_value

            # ランダムな揺れを加える（10~30%の揺れ）
            sales = base_sales * (1 + random.uniform(-0.3, 0.3))

            # 売上を整数に丸める
            sales = round(sales)

            # 現在の日時をUNIX時間のミリ秒に変換
            #timestamp_ms = int(current_date.timestamp() * 1000)

            # リストにUNIX時間と売上を追加
            sales_list.append([current_date, location, product, sales])

    # 1日進める
    current_date += datetime.timedelta(hours=1)

sales_data = pd.DataFrame(sales_list,columns=['time','region','measure_name','sales'])

上記のセルで2021/1/1から2024/4/30までの売り上げデータを作成しました。  
この売り上げデータから、時系列予測の学習モデルを作成するために用いる学習データを作成します。
学習データは2021/1/1から2024/4/23まで、推論用データは2024/4/1から2024/4/23までとし、2024/4/24から2024/4/30までを予測期間と設定します。

In [ ]:
train_data = sales_data.query('time <= "2024-04-23"')

# data ディレクトリがなければ作成
if not os.path.exists('data') :
    os.makedirs('data')

train_data_path='./data/train_data.csv'
train_data.to_csv(train_data_path, index=False)

In [ ]:
train_data.head(10)

## 3. 学習 <a name='training'>

AutoML トレーニングジョブ名を設定します。


In [ ]:
timestamp_suffix = strftime("%Y%m%d-%H%M%S", gmtime())
auto_ml_job_name = "ts-" + timestamp_suffix
print("AutoMLJobName: " + auto_ml_job_name)

トレーニングジョブの詳細を定義します。より詳細な情報は [AutoMLV2](https://sagemaker.readthedocs.io/en/v2.212.0/api/training/automlv2.html) をご覧ください。</n></n>  
この JSON はサンプルデータのスキーマについて示しています。独自のデータを用いる場合はドキュメントを参照し、パラメータを設定してください。  
以下のセルでSageMaker Autopilotで時系列予測を実行するために必要な設定を行います。トレーニングに使用するデータは1時間ごとのデータですが、今回は日次での予測を実行してみましょう。SageMaker Autopilot では時間頻度が異なるデータであっても集約の設定を行うことで内部で自動的に集約し、予測を生成することができます。

In [ ]:
from sagemaker.automl import automlv2

# 予測する頻度の設定
forecast_frequency = 'D'

# 予測するデータポイント数の設定
forecast_horizon = 7

# CSVファイルの各列の属性を設定
item_identifier_attribute_name = 'measure_name'
target_attribute_name = 'sales'
timestamp_attribute_name = 'time'
grouping_attribute_names = ['region']
# 1時間ごとのデータを日次に集約する際、合計値を算出するように設定
aggrecation ={'sales':'sum'}

# 予測結果で取得したい分位数を設定
forecast_quantiles = ['p10', 'p30', 'p50', 'p70', 'p90']

# 祝日のデータセットを用いるため国名コードを設定
holiday_config = 'JP'

# Autopilotで時系列予測を実行するための設定オブジェクトを作成
ts_config = automlv2.AutoMLTimeSeriesForecastingConfig(forecast_frequency=forecast_frequency,
                                                       forecast_horizon=forecast_horizon,
                                                       item_identifier_attribute_name=item_identifier_attribute_name,
                                                       target_attribute_name=target_attribute_name,
                                                       timestamp_attribute_name=timestamp_attribute_name,
                                                       grouping_attribute_names=grouping_attribute_names,
                                                       forecast_quantiles=forecast_quantiles,
                                                       holiday_config=holiday_config)

次にAutoMLクラスを作成し、学習を実行します。学習は約15分ほどで終了します。

In [ ]:
# S3にアップロードされるローカルにあるデータを表すクラス
inputs = automlv2.LocalAutoMLDataChannel('S3Prefix',train_data_path)

# 学習結果の出力S3先
output_path = 's3://{}/{}/train_output'.format(bucket, prefix)

# automlクラスの作成
automl = automlv2.AutoMLV2(ts_config,
                  base_job_name=auto_ml_job_name,
                  output_path=output_path,
                  role=role)

In [ ]:
# 学習を実行
automl.fit(inputs, job_name=auto_ml_job_name, wait=False, logs=False)

次のセルではジョブのステータスを監視します。ステータスが ```Completed``` になったらモデルの Accurancy を確認し、バッチ推論もしくはリアルタイム推論を実施するのかどうかを決定することができます。必要に応じて、 [AutoMLV2](https://sagemaker.readthedocs.io/en/v2.212.0/api/training/automlv2.html) のドキュメントを参照して下さい。

In [ ]:
# AutoML Job が終了するまで待つ
describe_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
job_run_status = describe_response["AutoMLJobStatus"]

if job_run_status not in ("Failed", "Completed", "Stopped"):
   print(datetime.datetime.now(), describe_response["AutoMLJobStatus"] + " - " + describe_response["AutoMLJobSecondaryStatus"])
   print('現在学習中です。学習が終了するまでお待ち下さい。')
elif job_run_status in ("Failed", "Stopped"):
   print(datetime.datetime.now(), describe_response["AutoMLJobStatus"] + " - " + describe_response["AutoMLJobSecondaryStatus"])
   print('学習が正常に終了しませんでした。ログを確認して下さい。')
elif job_run_status in "Completed":
   print(datetime.datetime.now(), describe_response["AutoMLJobStatus"] + " - " + describe_response["AutoMLJobSecondaryStatus"])
   print('学習が正常に終了しました。次のセルを実行して下さい。')


学習が完了したら、describe 関数を使ってモデルリーダーボードの結果を反復処理できます。以下は、この後の推論フェーズで最適な候補を使用する例となります。必要に応じて、 [create_model](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_model.html) にあるドキュメントを参照して下さい。

In [ ]:
# AutoML を実施した結果を確認
automl.describe_auto_ml_job(job_name=auto_ml_job_name)

In [ ]:
# AutoML で作成された最適なモデル名とコンテナイメージが格納されたECRのURLを取得する
best_candidate = automl.describe_auto_ml_job(job_name=auto_ml_job_name)['BestCandidate']
best_candidate_containers = best_candidate['InferenceContainers'] 
best_candidate_name = best_candidate['CandidateName']

print('BestCandidateName:',best_candidate_name)
print('BestCandidateContainers:',best_candidate_containers)

In [ ]:
# SageMaker にモデルを登録
reponse = sm.create_model(
    ModelName = best_candidate_name,
    ExecutionRoleArn = role,
    Containers = best_candidate_containers
)

## 4. 予測 <a name='forecast'>

本章では作成した学習モデルを用いて、時系列予測を実施します。

まずは推論に必要な CSV ファイルを作成します。
[SageMaker Autopilot のドキュメント](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/autopilot-create-experiment-timeseries-forecasting.html#autopilot-timeseries-forecasting-prerequisites) に記載されている通り、予測したいデータポイント1つにき、少なくとも3 - 5 個の履歴データポイントを提供することが推奨されています。  

今回は7日間の予測を取得しようとしているので、推論時に入力するデータには少なくとも21-35日前までのデータが含まれていることが推奨されます。  

2024/4/24から2024/4/30までの予測を実行しようとしているので、推論時に入力するデータは2024/4/1から2024/4/23までのデータを利用することとします。

In [ ]:
# 2024/4/1 - 4/23 までの
# region = location_a, 
# measure_name = product_a
# であるデータを抽出
inference_data = sales_data.query('"2024-04-01" <= time  and time <= "2024-04-23" and region == "location_a" and measure_name == "product_a"')

# 1時間ごとのデータを日次に変換
inference_data = pd.DataFrame(inference_data.set_index('time')['sales'].resample('D').sum()).assign(region="location_a",measure_name="product_a")
inference_path = './data/inference_data.csv'

# CSVファイルを出力
inference_data.to_csv(inference_path)

ここからは２種類の推論方法をご紹介します。

### 4.1.   リアルタイム推論を利用する場合<a name='realtime'>

リアルタイム予測を実行したい場合はこちらのセクションのセルを実行して下さい。もしバッチ予測を実行したい場合、このセクションを飛ばして [4.2 バッチ予測](#batch) を参照して下さい。 

リアルタイム予測を実現するために、SageMaker の Real-Time Inference というオプションを使ってエンドポイントを作成します。
Real-Time Inference はリアルタイム性、インタラクティブ性、低レイテンシでの応答が必要な場合に用いられるデプロイ方法です。

このセル以降で、候補となるメタデータを使ってモデル、エンドポイント構成、およびエンドポイント名を定義します。詳細は [create_endpoint_config](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_endpoint_config.html) ドキュメントを参照して下さい。また、必要に応じて ```InstanceType``` と ```InitialInstanceCount``` を設定して下さい。 

<span style="color: red; ">IMPORTANT</span>: 本ノートブックではサンプルのため、推論用インスタンスサイズに　```ml.m5.2xlarge``` を指定していますが、推奨は ```ml.m5.12xlarge``` となります。詳細は [ドキュメント](https://docs.aws.amazon.com/sagemaker/latest/dg/timeseries-forecasting-deploy-models.html) を参照して下さい。


In [ ]:
endpoint_config_name = f"epc-{best_candidate_name}"
endpoint_name = f"ep-{best_candidate_name}"

production_variants = [
        {
            "InstanceType": "ml.m5.2xlarge",
            "InitialInstanceCount": 1,
            "ModelName": best_candidate_name,
            "VariantName": "AllTraffic",
        }
    ]

epc_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=production_variants
)

次に [create_endpoint](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_endpoint.html
) API を使ってリアルタイム推論用のエンドポイントをデプロイすることができます。詳細やオプションについてはドキュメントを参照してください。

In [ ]:
sm.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name)

Status が InService になるまでエンドポイントをポーリングします。

In [ ]:
describe_response = sm.describe_endpoint(EndpointName=endpoint_name)

job_run_status = describe_response["EndpointStatus"]

while job_run_status not in ("Failed", "InService", "Stopped"):
    describe_response = sm.describe_endpoint(EndpointName=endpoint_name)
    job_run_status = describe_response["EndpointStatus"]

    print(datetime.datetime.now(), describe_response["EndpointStatus"])
    sleep(60)

推論用データを Python メモリオブジェクト上にロードします。

In [ ]:
f=open(inference_path,'r')
inference_data = f.read()
f.close()

SageMaker ランタイムクライアントをインスタンス化し、[invoke endpoint](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime/client/invoke_endpoint.html) する方法を以下に示します。ドキュメントに記載されているガイドには、エンドポイントの呼び出しから 60 秒以内に応答する必要があると記載されていることに注意して下さい。応答時間はペイロードサイズに依存します。本番環境でリアルタイム推論を実行する際は、単一の時系列データを使うようにして下さい。リアルタイム推論で一度に複数の時系列データの予測を実行したい場合は、応答時間を超えないように注意深くテストを実施してください。

In [ ]:
sm_client = boto3.client('sagemaker-runtime')

In [ ]:
response = sm_client.invoke_endpoint(
    EndpointName= endpoint_name,
    Body= inference_data,
    ContentType = 'text/csv')

prediction = response['Body'].read().decode()

この時点で、リアルタイム API からの呼び出し結果が ```prediction``` へロードされます。結果をどう処理するかについてはいくつか選択肢をとることができます。その例をいくつか示します。

Optional: リアルタイム予測結果をローカルファイルシステムに保存する例。命名規則は慎重に設定してください。

In [ ]:
output_file = './data/real-time-prediction-output.csv'
f=open(output_file,'w')
f.write(prediction)
f.close()

Optional: リアルタイム予測結果を S3 オブジェクトに保存する例。命名規則は慎重に設定してください。

In [ ]:
output_file = 'real-time-prediction-output.csv'
key='s3://{}/{}/realtime_inference/{}'.format(bucket, prefix, output_file)

s3_client = boto3.client('s3')
s3_client.put_object(Body=prediction, Bucket=bucket, Key=key)

Optional: リアルタイム予測結果を Pandas の DataFrame 化する例

In [ ]:
import pandas as pd
from io import StringIO
df = pd.read_csv(StringIO(prediction), sep=',')
df.head(5)

最終的に必要に応じて、予測を SQS キューに入れたり、ストリームに投稿したり、別のリアルタイム API に POST することも可能です。

#### 予測結果の可視化  
では、予測した結果を可視化してみましょう。以下のセルでは、ある一つの商品に着目し、予測結果と実データを可視化します。

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
%matplotlib inline

In [ ]:
FROM_DATE = pd.to_datetime("2024-04-01 00:00:00",utc=True)
TO_DATE = pd.to_datetime("2024-04-30 23:30:00",utc=True)
product_location_a = sales_data[(sales_data['measure_name'] == 'product_a') &(sales_data['region']=='location_a')].query('time >= "2024-04-01"')
product_location_a = product_location_a.set_index('time')
# 1時間ごとのデータを日次に変換
product_location_a = product_location_a.resample('D').sum()['sales']

In [ ]:
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
results = df[(df['measure_name'] == 'product_a') &(df['region']=='location_a')]

In [ ]:
color_a = 'C0'
color_b = 'C1'
color_c = 'C2'
color_d = 'C3'

fig, ax1 = plt.subplots()
ax1.set_title("Forecast TTS only", fontsize = 14)
ax1.plot(product_location_a, color=color_a)
ax1.plot(results.loc[:,"p50"],color=color_b, alpha=0.4)
ax1.plot(results.loc[:,"p90"],color=color_c, alpha=0.4)
ax1.plot(results.loc[:,"p10"],color=color_d, alpha=0.4)

ax1.grid(which="major", axis="both")
fig.set_figwidth(15)
fig.set_figheight(10)

時系列予測の精度を確認する際は、予測した期間のデータとその期間の実データと照らし合わせることが重要です。

#### リアルタイムエンドポイントを削除する

必要に応じて、エンドポイントを削除して下さい、再度エンドポイントをデプロイする必要になった場合は、再度本節のステップに従いコードを実行してください。その場合、別の新しいモデルをトレーニングしてデプロイすることが望ましいです。

In [ ]:
sm.delete_endpoint(EndpointName=endpoint_name)

In [ ]:
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

<span style="color: red; ">CAUTION:</span> バッチ変換も実行する予定の場合は、以下のセルを実行してモデルを削除しないようにしてください。モデルを削除してもモデルアーティファクトが S3 にある限り、モデルを再デプロイすることは可能です。次のセルを実行することで最適な候補モデルのコンテナと S3 の場所が表示されます。

In [ ]:
print('BestCandidateContainers:',best_candidate_containers)

In [ ]:
sm.delete_model(ModelName=best_candidate_name)

### 4.2 バッチ推論を利用する場合<a name='batch'>

[service limits](https://docs.aws.amazon.com/marketplace/latest/userguide/ml-service-restrictions-and-limits.html) ページで Batch Transform に関連する箇所を確認して下さい。2024/02/06 現在、ドキュメントには１回の呼び出しあたりの入力データの最大サイズは 100MB であると記載されています。データセットが 100MB を超える場合は複数のファイルに分割/シャーディングして下さい。Item ID などでデータを分割し、各ファイルに時系列データの全体が含まれていることを確認してください。

[create_transform_job] (https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_transform_job.html) を使用して Batch Tranform ジョブを起動します。ジョブの実行時間はデータサイズやインスタンスタイプ、インスタンス数に依存します。タスクが完了すると、実行結果は ```S3OutputPath``` で指定された場所にデータが出力されます。これより、Event hundler やその他のメカニズムを利用して予測結果を処理することができます。

In [ ]:
timestamp_suffix = strftime("%Y%m%d-%H%M%S", gmtime())
transform_job_name=f'{best_candidate_name}-' + timestamp_suffix
print("BatchTransformJob: " + transform_job_name)

次のセルはデータセットを再度ダウンロードし、```batch_transform/input``` フォルダにデータセットを格納します。

<span style="color: red; ">IMPORTANT:</span> 推論のために提供するデータは、各時系列について少なくとも4つの有効な履歴値を持っていなければなりません。

In [ ]:
s3 = boto3.client('s3')
object_inference_key = '{}/batch_transform/input/batch_transform.csv'.format(prefix)
s3.upload_file(inference_path, bucket, object_inference_key)

In [ ]:
response = sm.create_transform_job(
    TransformJobName=transform_job_name, 
    ModelName=best_candidate_name,
    MaxPayloadInMB=0,
    ModelClientConfig={
        'InvocationsTimeoutInSeconds': 3600
    },
    TransformInput={
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': 's3://{}/{}/batch_transform/input/'.format(bucket, prefix)
            }
        },
        'ContentType': 'text/csv',
        'SplitType': 'None'
    },
    TransformOutput={
        'S3OutputPath': 's3://{}/{}/batch_transform/output/'.format(bucket, prefix),
        'AssembleWith': 'Line',
    },
    TransformResources={
        'InstanceType': 'ml.m5.12xlarge',
        'InstanceCount': 1
    }
    )

バッチ変換ジョブが完了するまでポーリングします。完了すると、結果の予測ファイルは前のセルで示したURI ``S3OutputPath`` で利用できるようになります。このステップを完了するために、APIメソッド[describe_transform_job](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/describe_transform_job.html)を使用します。

In [ ]:
describe_response = sm.describe_transform_job(TransformJobName=transform_job_name)

job_run_status = describe_response["TransformJobStatus"]

while job_run_status not in ("Failed", "Completed", "Stopped"):
    describe_response = sm.describe_transform_job(TransformJobName=transform_job_name)
    job_run_status = describe_response["TransformJobStatus"]

    print(
       datetime.datetime.now(), describe_response["TransformJobStatus"]
    )
    sleep(60)